In [4]:
#Lab 4: Integrating and Visualizing Disparate Data Sources

import shapefile
import pandas
from sklearn import datasets, linear_model
import matplotlib.pyplot as plt
import numpy as np
import seaborn
%matplotlib inline 
import random
import multiprocessing
import scipy

#If things aren't working, disable this!!!:
import warnings
warnings.filterwarnings("ignore")

In [5]:
#loading in my dataset
cameroon_dta = pandas.read_csv("results_v2.csv")
cameroon_dta

asdf_id  africa_child_mortality.1990.mean  \
0          0                         31.878841   
1          1                         33.922019   
2          2                         37.738651   
3          3                         42.869401   
4          4                         32.912441   
5          5                         33.385189   
6          6                         39.440191   
7          7                         30.204584   
8          8                         42.793196   
9          9                         43.538213   
10        10                         48.066699   
11        11                         36.393506   
12        12                         29.781927   
13        13                         33.806057   
14        14                         28.954675   
15        15                         24.927498   
16        16                         27.015244   
17        17                         25.135528   
18        18                         35.591797   
19        19                         22.226382   
20        20                         28.086378   
21        21                         26.861309   
22        22                         29.968497   
23        23                         24.962080   
24        24                         23.666986   
25        25                         18.105012   
26        26                         37.832657   
27        27                         34.002281   
28        28                         30.570614   
29        29                         26.112483   
..       ...                               ...   
330      330                         41.673448   
331      331                         27.461997   
332      332                         45.743006   
333      333                         27.117488   
334      334                         29.297839   
335      335                         54.550789   
336      336                         32.103589   
337      337                         29.472243   
338      338                         26.556234   
339      339                         29.274166   
340      340                         29.293853   
341      341                         29.910404   
342      342                         29.183145   
343      343                         30.905065   
344      344                         28.473216   
345      345                         30.242585   
346      346                         28.156986   
347      347                         29.380483   
348      348                         29.689920   
349      349                         29.661063   
350      350                         30.480518   
351      351                         30.137843   
352      352                         28.769607   
353      353                         31.451413   
354      354                         30.441825   
355      355                         30.190282   
356      356                         29.082646   
357      357                         22.632101   
358      358                         29.148445   
359      359                         29.328902   

     africa_child_mortality.1980.mean  africa_child_mortality.2000.mean  \
0                           33.338962                         31.553367   
1                           34.003932                         32.710375   
2                           35.376072                         32.719621   
3                           38.400910                         34.833894   
4                           32.132439                         32.602583   
5                           27.974978                         22.291583   
6                           36.343824                         29.201398   
7                           29.333157                         21.665656   
8                           38.628787                         32.293667   
9                           48.973940                         36.708872   
10                          40.382686                         36.116800   
11  

In [10]:
#For my model, I have selected to predict child mortality based on population, 
#urban access, and nighttime lights.  To interpret what the numbers in the above table mean,
#take a careful look at the metadata you retrieve.
#You are expected to add additional columns of data for this analysis, retrieving your own data
#from www.geoquery.org .
#First, I'm going to build a predictive model based on the past (note, one thing
#is intentionally theoretically wrong with this model, AND you can do much better.
#Think carefully about what variables you use!!):

cameroon_dta = pandas.read_csv("results_v2.csv")
cameroon_dta_NoNA = cameroon_dta[['africa_child_mortality.1990.mean',
                              'gpw_v3_count.1995.sum',
                              'access_50k.none.mean',
                              'v4composites_calibrated_201709.1999.mean',
                              'udel_air_temp_v401_mean.1995.mean',
                              'udel_precip_mean.1995.mean']].dropna()
length = len(cameroon_dta_NoNA)
outcome = cameroon_dta_NoNA['africa_child_mortality.1990.mean'].values.reshape(length, 1)

#I want to introduce uncertainty into Nighttime Lights and population.
#You will want to use literature to justify the error you add,
#with citations!
for i in range(len(cameroon_dta_NoNA)):

    lower_bound = cameroon_dta_NoNA.iloc[i]['v4composites_calibrated_201709.1999.mean'] * 0.85
    upper_bound = cameroon_dta_NoNA.iloc[i]['v4composites_calibrated_201709.1999.mean'] * 1.15
    if(lower_bound < 0):
        lower_bound = 0

    #Read your metadata!
    if(upper_bound > 63):
        upper_bound = 63

    cameroon_dta_NoNA.iloc[i]['v4composites_calibrated_201709.1999.mean'] = random.uniform(lower_bound,upper_bound)

    lower_bound = cameroon_dta_NoNA.iloc[i]['gpw_v3_count.1995.sum'] * 0.5
    upper_bound = cameroon_dta_NoNA.iloc[i]['gpw_v3_count.1995.sum'] * 1.5
    if(lower_bound < 0):
        lower_bound = 0

    #Read your metadata!
    if(upper_bound > 250000):
        upper_bound = 250000       

    cameroon_dta_NoNA.iloc[i]['gpw_v3_count.1995.sum'] = random.uniform(lower_bound,upper_bound)
    
    lower_bound = cameroon_dta_NoNA.iloc[i]['udel_air_temp_v401_mean.1995.mean'] * 0.3
    upper_bound = cameroon_dta_NoNA.iloc[i]['udel_air_temp_v401_mean.1995.mean'] * 1.7
    if(lower_bound < 0):
        lower_bound = 0

    #Read your metadata!
    if(upper_bound > 40):
        upper_bound = 40       

    cameroon_dta_NoNA.iloc[i]['udel_air_temp_v401_mean.1995.mean'] = random.uniform(lower_bound,upper_bound)
    
    lower_bound = cameroon_dta_NoNA.iloc[i]['udel_precip_mean.1995.mean'] * 0.85
    upper_bound = cameroon_dta_NoNA.iloc[i]['udel_precip_mean.1995.mean'] * 1.15
    if(lower_bound < 0):
        lower_bound = 0

    #Read your metadata!
    if(upper_bound > 300):
        upper_bound = 300         

    cameroon_dta_NoNA.iloc[i]['udel_precip_mean.1995.mean'] = random.uniform(lower_bound,upper_bound)    

predictors = cameroon_dta_NoNA[['gpw_v3_count.1995.sum',
                           'access_50k.none.mean',
                           'v4composites_calibrated_201709.1999.mean',
                           'udel_air_temp_v401_mean.1995.mean',
                           'udel_precip_mean.1995.mean']].values.reshape(length, 5)        

historic_model = linear_model.LinearRegression()
historic_model.fit(predictors, outcome)  
estimated_1990_mortality = historic_model.predict(predictors)

MAE = np.mean(np.abs(estimated_1990_mortality - cameroon_dta_NoNA['africa_child_mortality.1990.mean'].values))
    
print("Model MAE:")
print(MAE)
print("Model Coefficients:")
print(historic_model.coef_)

Model MAE:
10.4096658342
Model Coefficients:
[[  5.49663568e-05   1.17106690e-02  -2.89085328e-01   6.81785501e-02
   -9.28627227e-02]]


In [5]:
#In my case, I know how well my model performs in 1990, based on 1990s data - i.e.,
#I can get values accurate to within approximately 9.5 under-5 mortality events.
#You are expected to do better than this - there are *many* sources of data
#not currently being used that you can draw on.
#Treat my model as the "baseline" model - i.e., the first, easiest model
#you could run. 

#I'm now going to use my baseline model to project into the future. 
#To do this, I need future estimates of data I would not have in 1999:
#Population,
#Urban Access,
#Nighttime Lights
cameroon_dta_predictions = cameroon_dta[['africa_child_mortality.1990.mean',
                              'gpw_v3_count.1995.sum',
                              'access_50k.none.mean',
                              'v4composites_calibrated_201709.1999.mean',
                              'udel_air_temp_v401_mean.1995.mean',
                              'udel_precip_mean.1995.mean',
                              'africa_child_mortality.2000.mean',
                              'NAME_3']].dropna()

#There is no meaningful way for me to get new data for urban access, so I will use
#the same data for 2000 in 1990, and would discuss this in my limitations.  I.e.:
#"While it is known that urban accessibility has an impact on child mortatlity,
#no updated information is available that provides measurements for the year 2000.
#Following this, this analysis uses the same input data for both calibration and prediction."

#For population, I have two points in time - 1995 and 1990. 
#I will use the advanced "draw a straight line to 2000" technique to predict
#what population will be.  There are much, much more complex demographic models you can employ;
#I intentionally omit them here for simplicity.
cameroon_dta_predictions['2000_pop'] = (cameroon_dta['gpw_v3_count.1995.sum'] + 
                                        5*((cameroon_dta['gpw_v3_count.1995.sum'] - 
                                            cameroon_dta['gpw_v3_count.1990.sum'])/6))


#Same thing for NTL. You can of course do much better than this straight line!
cameroon_dta_predictions['2000_ntl'] = (cameroon_dta['v4composites_calibrated_201709.1999.mean'] + 
                                        1*((cameroon_dta['v4composites_calibrated_201709.1999.mean'] - 
                                            cameroon_dta['v4composites_calibrated_201709.1992.mean'])/8))

cameroon_dta_predictions['2000_air_temp'] = (cameroon_dta['udel_air_temp_v401_mean.1995.mean'] + 
                                        5*((cameroon_dta['udel_air_temp_v401_mean.1995.mean'] - 
                                            cameroon_dta['udel_air_temp_v401_mean.1990.mean'])/6))

cameroon_dta_predictions['2000_precip'] = (cameroon_dta['udel_precip_mean.1995.mean'] + 
                                        5*((cameroon_dta['udel_precip_mean.1995.mean'] - 
                                            cameroon_dta['udel_precip_mean.1990.mean'])/6))

#Now, I'm going to create my future predictors dataset using these new values.
predictors_millenium = cameroon_dta_predictions[['2000_pop',
                           'access_50k.none.mean',
                           '2000_ntl',
                           '2000_air_temp',
                           '2000_precip']].values.reshape(length, 5)

#And, finally, predict the future.
child_mortality_2000_prediction =  historic_model.predict(predictors_millenium)
cameroon_dta_predictions['2000_pred'] = child_mortality_2000_prediction
#How accurate was this prediction based on real data?:
abs_diff_pred = np.abs(child_mortality_2000_prediction - cameroon_dta_predictions['africa_child_mortality.2000.mean'].values)
seaborn.distplot(abs_diff_pred[0])
MAE_pred = np.mean(np.abs(child_mortality_2000_prediction - cameroon_dta_predictions['africa_child_mortality.2000.mean'].values))
print("Prediction MAE:")
print(MAE_pred)


#Finally, I want to estimate the number of units of observation for which I
#correctly estimated the true value within 10%, 20%, and 30% of the truth,
#and highlight the top 10 areas with the worst rates of child mortality.
tenPercent = abs_diff_pred <= cameroon_dta_predictions['africa_child_mortality.2000.mean'].values * 0.1
twentyPercent = abs_diff_pred <= cameroon_dta_predictions['africa_child_mortality.2000.mean'].values * 0.2
thirtyPercent = abs_diff_pred <= cameroon_dta_predictions['africa_child_mortality.2000.mean'].values * 0.3
print("Predictions within 10, 20, and 30%")
print(np.mean(tenPercent))
print(np.mean(twentyPercent))
print(np.mean(thirtyPercent))

top = cameroon_dta_predictions.nlargest(10, '2000_pred')
print("Top 10 Districts in Cameroon, Predicted Child Mortality, 2000; Observed, 1990:")
print(top[["NAME_3",'2000_pred','africa_child_mortality.1990.mean']])

Prediction MAE:
10.3690062626
Predictions within 10, 20, and 30%
0.171592646921
0.343505196467
0.494444617833
Top 10 Districts in Cameroon, Predicted Child Mortality, 2000; Observed, 1990:
         NAME_3  2000_pred  africa_child_mortality.1990.mean
90    Moloundou  52.065373                         44.111869
213         Ako  49.278644                         30.166783
91   Salapoumbé  48.599359                         45.592312
48         Yoko  45.791044                         26.142254
104      Ngoïla  45.674780                         48.794720
335    Mintom 2  45.114884                         54.550789
332       Djoum  44.524836                         45.743006
131    Blangoua  44.172211                         38.686186
134  Hilé Alifa  44.157339                         42.099331
99        Lomié  43.877781                         46.244379


In [1]:
#####Putting it all together, with uncertainty this time!#######
#As always in this course, we don't believe our data is perfect.
#As one example, I want to test my assumption of linear population and NTL
#growth.  I am also going to assume my measurements of NTL aren't perfect,
#but are accurate to within 10% of the true value.
#I base this on desk research into Nighttime Lights 
#(Remote Sensing and GIS Accuracy Assesment, 20.4 - Conclusions),
#Indicates a R2 value of .9812 of NTL vs. other data, suggesting a high
#degree of accuracy.  I round this down to 90% accuracy to make for
# a very conservative test.
#Note this code intentionally has a number of small issues -i.e.,
#it is possible to estimate errors that result in negative populations
#as it is written.  Make sure to fix these.
#How big of a difference could introducing this uncertainty make?

import shapefile
import pandas
from sklearn import datasets, linear_model
import matplotlib.pyplot as plt
import numpy as np
import seaborn
%matplotlib inline 
import random
import multiprocessing
import scipy

#If things aren't working, disable this!!!:
import warnings
warnings.filterwarnings("ignore")



def underFiveMortality(worker_id):
    print("Worker " + str(worker_id) + " starting.")
    scipy.random.seed()
    #Historic model - 
    cameroon_dta = pandas.read_csv("results_v2.csv")
    cameroon_dta_NoNA = cameroon_dta[['africa_child_mortality.1990.mean',
                              'gpw_v3_count.1995.sum',
                              'access_50k.none.mean',
                              'v4composites_calibrated_201709.1999.mean',
                              'udel_air_temp_v401_mean.1995.mean',
                              'udel_precip_mean.1995.mean']].dropna()

    length = len(cameroon_dta_NoNA)

    outcome = cameroon_dta_NoNA['africa_child_mortality.1990.mean'].values.reshape(length, 1)

    #I want to introduce uncertainty into Nighttime Lights and population.
    #You will want to use literature to justify the error you add,
    #with citations!
    for i in range(len(cameroon_dta_NoNA)):
        
        lower_bound = cameroon_dta_NoNA.iloc[i]['v4composites_calibrated_201709.1999.mean'] * 0.9
        upper_bound = cameroon_dta_NoNA.iloc[i]['v4composites_calibrated_201709.1999.mean'] * 1.1
        if(lower_bound < 0):
            lower_bound = 0
        
        #Read your metadata!
        if(upper_bound > 63):
            upper_bound = 63
            
        cameroon_dta_NoNA.iloc[i]['v4composites_calibrated_201709.1999.mean'] = random.uniform(lower_bound,upper_bound)

        lower_bound = cameroon_dta_NoNA.iloc[i]['gpw_v3_count.1995.sum'] * 0.5
        upper_bound = cameroon_dta_NoNA.iloc[i]['gpw_v3_count.1995.sum'] * 1.5
        if(lower_bound < 0):
            lower_bound = 0
        
        #Read your metadata!
        if(upper_bound > 250000):
            upper_bound = 250000                

        cameroon_dta_NoNA.iloc[i]['gpw_v3_count.1995.sum'] = random.uniform(lower_bound,upper_bound)

        lower_bound = cameroon_dta_NoNA.iloc[i]['udel_air_temp_v401_mean.1995.mean'] * 0.3
        upper_bound = cameroon_dta_NoNA.iloc[i]['udel_air_temp_v401_mean.1995.mean'] * 1.7
        if(lower_bound < 0):
            lower_bound = 0
        
        #Read your metadata!
        if(upper_bound > 40):
            upper_bound = 40        

        cameroon_dta_NoNA.iloc[i]['udel_air_temp_v401_mean.1995.mean'] = random.uniform(lower_bound,upper_bound)

        lower_bound = cameroon_dta_NoNA.iloc[i]['udel_precip_mean.1995.mean'] * 0.85
        upper_bound = cameroon_dta_NoNA.iloc[i]['udel_precip_mean.1995.mean'] * 1.15
        if(lower_bound < 0):
            lower_bound = 0
        
        #Read your metadata!
        if(upper_bound > 300):
            upper_bound = 300        

        cameroon_dta_NoNA.iloc[i]['udel_precip_mean.1995.mean'] = random.uniform(lower_bound,upper_bound)   

    predictors = cameroon_dta_NoNA[['gpw_v3_count.1995.sum',
                               'access_50k.none.mean',
                               'v4composites_calibrated_201709.1999.mean',
                               'udel_air_temp_v401_mean.1995.mean',
                               'udel_precip_mean.1995.mean']].values.reshape(length, 5)    

    historic_model = linear_model.LinearRegression()

    historic_model.fit(predictors, outcome)


    #Future Predictions
    cameroon_dta_predictions = cameroon_dta[['africa_child_mortality.1990.mean',
                                  'gpw_v3_count.1995.sum',
                                  'gpw_v3_count.1990.sum',
                                  'access_50k.none.mean',
                                  'v4composites_calibrated_201709.1999.mean',
                                  'v4composites_calibrated_201709.1992.mean',
                                  'udel_air_temp_v401_mean.1995.mean',
                                  'udel_air_temp_v401_mean.1990.mean',
                                  'udel_precip_mean.1995.mean',
                                  'udel_precip_mean.1990.mean',
                                  'africa_child_mortality.2000.mean',
                                  'NAME_3']].copy(deep=True).dropna()

    #I want to introduce considerable uncertainty into this population estimate.
    #From desk research, I have found that population estimates in Cameroon are highly variable,
    #ranging from +25 to -25%.  Because I want to be particularly conservative in my estimate
    #due to my own uncertainty if linear models are correct, I'm going to round this up to
    #+/- 30%.
    cameroon_dta_predictions['2000_pop'] = (cameroon_dta_predictions['gpw_v3_count.1995.sum'] + 
                                        5*((cameroon_dta_predictions['gpw_v3_count.1995.sum'] - 
                                            cameroon_dta_predictions['gpw_v3_count.1990.sum'])/6))

    for i in range(len(cameroon_dta_predictions)):
        lower_bound = cameroon_dta_predictions.iloc[i]['2000_pop'] * 0.3
        upper_bound = cameroon_dta_predictions.iloc[i]['2000_pop'] * 1.7
        
            
        cameroon_dta_predictions.iloc[i]['2000_pop'] = random.uniform(lower_bound,upper_bound)
    
        #I am adding a lot of uncertainty to the urban access metric, representing
        #very little confidence in the estimate.
        lower_bound = cameroon_dta_predictions.iloc[i]['access_50k.none.mean'] * 0.0
        upper_bound = cameroon_dta_predictions.iloc[i]['access_50k.none.mean'] * 1.5
        

        cameroon_dta_predictions.iloc[i]['access_50k.none.mean'] = random.uniform(lower_bound,upper_bound)

    #Same thing for NTL.  To be conservative, I am adding an additional 30% of 
    #uncertainty to the original 10% in the historic model.
    cameroon_dta_predictions['2000_ntl'] = (cameroon_dta_predictions['v4composites_calibrated_201709.1999.mean'] + 
                                            1*((cameroon_dta_predictions['v4composites_calibrated_201709.1999.mean'] - 
                                                cameroon_dta_predictions['v4composites_calibrated_201709.1992.mean'])/8))

    for i in range(len(cameroon_dta_predictions)):
        mean = cameroon_dta_predictions.iloc[i]['2000_ntl']
        lower_bound = cameroon_dta_predictions.iloc[i]['2000_ntl'] * 0.5
        upper_bound = cameroon_dta_predictions.iloc[i]['2000_ntl'] * 1.5

            
        cameroon_dta_predictions.iloc[i]['2000_ntl'] = random.uniform(lower_bound,upper_bound)
        
    #Same thing for air temp.
    
    cameroon_dta_predictions['2000_air_temp'] = (cameroon_dta['udel_air_temp_v401_mean.1995.mean'] + 
                                            5*((cameroon_dta['udel_air_temp_v401_mean.1995.mean'] - 
                                                cameroon_dta['udel_air_temp_v401_mean.1990.mean'])/6))        

    for i in range(len(cameroon_dta_predictions)):
        mean = cameroon_dta_predictions.iloc[i]['2000_air_temp']
        lower_bound = cameroon_dta_predictions.iloc[i]['2000_air_temp'] * 0.85
        upper_bound = cameroon_dta_predictions.iloc[i]['2000_air_temp'] * 1.15 
        
        
        cameroon_dta_predictions.iloc[i]['2000_air_temp'] = random.uniform(lower_bound,upper_bound)
    
    #Same thing for precip.
    
    cameroon_dta_predictions['2000_precip'] = (cameroon_dta['udel_precip_mean.1995.mean'] + 
                                            5*((cameroon_dta['udel_precip_mean.1995.mean'] - 
                                                cameroon_dta['udel_precip_mean.1990.mean'])/6))        

    for i in range(len(cameroon_dta_predictions)):
        mean = cameroon_dta_predictions.iloc[i]['2000_precip']
        lower_bound = cameroon_dta_predictions.iloc[i]['2000_precip'] * 0.85
        upper_bound = cameroon_dta_predictions.iloc[i]['2000_precip'] * 1.15 
        
        
        cameroon_dta_predictions.iloc[i]['2000_precip'] = random.uniform(lower_bound,upper_bound)    
    
    child_mortality_2000_prediction =  (historic_model.coef_[0][0] * cameroon_dta_predictions['2000_pop'].values +
                                        historic_model.coef_[0][1] * cameroon_dta_predictions['access_50k.none.mean'].values +
                                        historic_model.coef_[0][2] * cameroon_dta_predictions['2000_ntl'].values +
                                        historic_model.coef_[0][3] * cameroon_dta_predictions['2000_air_temp'].values +
                                        historic_model.coef_[0][4] * cameroon_dta_predictions['2000_precip'].values)
        
    #I'm going to save the top 10 locations, and my overall accuracy
    #at the 20% threshold (i.e., number of estimates I get within 20%)
    #based on the true 2000 data.
    abs_diff_pred = np.abs(child_mortality_2000_prediction - cameroon_dta_predictions['africa_child_mortality.2000.mean'].values)
    twentyPercent = np.mean(abs_diff_pred <= 
                            cameroon_dta_predictions['africa_child_mortality.2000.mean'].values * 0.2)
    
    cameroon_dta_predictions['2000_pred'] = child_mortality_2000_prediction
    top = cameroon_dta_predictions.nlargest(10, '2000_pred')
    
    write_out = open("./sim_res/result_v2_"+str(random.random())+".csv",mode='w')
    for i in cameroon_dta_predictions.loc[cameroon_dta_predictions["NAME_3"].isin(top["NAME_3"].values)].NAME_3.values:
        write_out.write(i + ",")
    write_out.write(str(np.mean(twentyPercent)))
    write_out.close()
    
    print("Worker " + str(worker_id) + " finished.")
    
iterations = 10

print("You have access to a maximum of " + str(multiprocessing.cpu_count()*2)+ " threads.")
pool = multiprocessing.Pool(multiprocessing.cpu_count()*2-2)

result = pool.map(underFiveMortality, range(iterations))

You have access to a maximum of 16 threads.
Worker 0 starting.
Worker 3 starting.
Worker 5 starting.
Worker 1 starting.
Worker 2 starting.
Worker 4 starting.
Worker 6 starting.
Worker 9 starting.
Worker 8 starting.
Worker 7 starting.
Worker 0 finished.
Worker 7 finished.
Worker 6 finished.
Worker 3 finished.
Worker 8 finished.
Worker 4 finished.
Worker 9 finished.
Worker 1 finished.
Worker 5 finished.
Worker 2 finished.


In [2]:
import glob
import os   
import pandas
import collections
import seaborn
import matplotlib.pyplot as plt
%matplotlib inline 


all_files = glob.glob(os.path.join('./sim_res/', "*.csv"))   
df_set = (pandas.read_csv(f, header=None) for f in all_files)
all_simulation_results = pandas.concat(df_set, ignore_index=True)

accuracy_metric = all_simulation_results[10].values
all_simulation_results = all_simulation_results.drop(all_simulation_results.columns[10], axis=1)
print(accuracy_metric)

#Now, let's create our bar chart telling us how many times each one was picked.
flat_list = [item for sublist in all_simulation_results.values for item in sublist]
noNAList = [x for x in flat_list if str(x) != 'nan']
district_counts = collections.Counter(noNAList)
plt.bar(range(len(district_counts)), list(district_counts.values()), align='center')
plt.xticks(range(len(district_counts)), list(district_counts.keys()), rotation="vertical")
plt.ylabel("Number of Iterations in Top 10")
plt.xlabel("District Name")
plt.title("Districts in Top 10")
plt.show()

[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.]


In [3]:
seaborn.distplot(accuracy_metric, norm_hist=False, kde=False)